In [3]:
import torch_geometric.nn as gnn
import torch_geometric
import torch
import torch.nn as nn
from torch_geometric.datasets import Planetoid
import torch.nn.functional as F
import scipy.linalg
from torch_geometric.nn import GCNConv

In [4]:
name_data = 'Cora'
dataset = Planetoid(root= '/tmp/' + name_data, name = name_data)
data = dataset[0]

edges = data.edge_index

#dimension: num_nodes * num_features
features = data.x

num_nodes = features.shape[0]
num_features = features.shape[1]
#dimension: num_nodes * num_nodes
adj = torch.zeros((num_nodes,num_nodes))
degree = torch.zeros((num_nodes,num_nodes))
for i in range(0, edges.shape[1]):
    first = edges[0][i]
    second = edges[1][i]
    adj[first][second] = 1
for i in range(0, num_nodes):
    degree[i][i] = sum(adj[i][:])
lap = degree - adj
d_inv = scipy.linalg.fractional_matrix_power(degree, (-1/2))
d_inv = torch.from_numpy(d_inv)
Llp = torch.mm(torch.mm(d_inv, lap), d_inv)


In [ ]:
print(Llp.shape)

1433


In [ ]:
print(torch.transpose(features,0, 1).shape)
print(lap.shape)
EGS_feature = torch.trace(torch.mm(torch.mm(torch.transpose(features,0, 1), Llp), features))
EG_feature = torch.trace(torch.mm(torch.transpose(features,0, 1), features))
S_feature = EGS_feature / EG_feature
print(S_feature)

lap_l = lap.long()
label = data.y.long()
EGS_label = torch.matmul(torch.matmul(label, lap_l), label)
EG_label = torch.matmul(label, label)
S_label = EGS_label / EG_label
print(S_label)



torch.Size([1433, 2708])
torch.Size([2708, 2708])
tensor(0.8451)
tensor(0.3231)


In [5]:
class FBLayer(nn.Module):
    def __init__(self, dim_in, dim_out):
      super().__init__()
      self.low = nn.Linear(dim_in, dim_out, bias = False)
      self.high = nn.Linear(dim_in, dim_out, bias = False)
      self.aL = torch.nn.Parameter(torch.rand(1))
      self.aH = torch.nn.Parameter(torch.rand(1))
      gain = nn.init.calculate_gain("relu")
      nn.init.xavier_normal_(self.low.weight, gain)
      nn.init.xavier_normal_(self.high.weight, gain)
      self.activation = nn.ReLU()
        
    def forward(self, input, adjacency, degree):
      lap = degree - adjacency
      d_inv = torch.from_numpy(scipy.linalg.fractional_matrix_power(degree, (-1/2)))
      Llp = torch.mm(torch.mm(d_inv, lap), d_inv)
      Lhp = torch.eye(input.shape[0]) - Llp
      Hl = torch.mm(Llp, self.activation(self.low(input)))
      Hh = torch.mm(Lhp, self.activation(self.high(input)))
      H = self.aL * Hl + self.aH * Hh
      return H

In [ ]:
class FB(nn.Module):
    def __init__(self, dim_in, dim_out):
      super().__init__()
      self.layer1 = FBLayer(dim_in, 128)
      self.layer2 = FBLayer(128, 64)
      self.layer3 = FBLayer(64, dim_out)
    def forward(self, input, adjacency, degree):
      ret = self.layer1(input, adjacency, degree)
      ret = self.layer2(ret, adjacency, degree)
      ret = self.layer3(ret, adjacency,degree)
      return F.log_softmax(ret, dim=1)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.conv1 = GCNConv(dataset.num_node_features, 16)
      self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, input, edge):
      x = input
      edge_index = edge
      x = self.conv1(x, edge_index)
      x = F.relu(x)
      x = F.dropout(x, training=self.training)
      x = self.conv2(x, edge_index)

      return F.log_softmax(x, dim=1)

In [6]:
import numpy as np
sample_num = int(num_nodes*0.6)

sample_lst = np.random.choice(num_nodes,sample_num, replace = False)
train_mask = [False] * num_nodes
test_mask = [True] * num_nodes
for i in sample_lst:
  train_mask[i] = True
  test_mask[i] = False
train_mask = torch.tensor(train_mask)
test_mask = torch.tensor(test_mask)
print(train_mask.sum().item())



1624


In [7]:
class FBGCN(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.fb1 = FBLayer(dataset.num_node_features,dataset.num_node_features)
      self.fb2 = FBLayer(dataset.num_node_features,dataset.num_node_features)
      self.conv1 = GCNConv(dataset.num_node_features, 16)
      self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data, adjacency, degree):
      input, edge_index = data.x, data.edge_index
      x = self.fb1(input, adjacency, degree)
      x = self.fb2(x, adjacency, degree)
      x = self.conv1(x, edge_index)
      x = F.relu(x)
      x = F.dropout(x, training=self.training)
      x = self.conv2(x, edge_index)

      return F.log_softmax(x, dim=1)


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
model1 = GCN().to(device)
optimizer = torch.optim.Adam(model1.parameters(), lr=0.05, weight_decay=5e-5)
total = 0
test_num = 1
for i in range(0,test_num):
  model1.train()
  for epoch in range(100):
      optimizer.zero_grad()
      out = model1(data.x, data.edge_index)
      loss = F.nll_loss(out[train_mask], data.y[train_mask])
      loss.backward()
      optimizer.step()

  model1.eval()
  _, pred = model1(data.x, data.edge_index).max(dim=1)
  correct = int(pred[test_mask].eq(data.y[test_mask]).sum().item())
  acc = correct / int(test_mask.sum())
  total += acc
total = total/test_num
print('GCN only Accuracy: {:.4f}'.format(total))

NameError: name 'GCN' is not defined

In [ ]:
model2 = FB(num_features, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model2.parameters(), lr=0.05, weight_decay=5e-5)
features = features.to(device)
adj = adj.to(device)
degree = degree.to(device)
total = 0
test_num = 1
for i in range(0,test_num):
  model2.train()
  for epoch in range(50):
      optimizer.zero_grad()
      out = model2(features, adj, degree)
      loss = F.nll_loss(out[train_mask], data.y[train_mask])
      loss.backward()
      optimizer.step()
  model2.eval()
  _, pred = model2(features, adj, degree).max(dim=1)
  correct = int(pred[test_mask].eq(data.y[test_mask]).sum().item())
  acc = correct / int(test_mask.sum())
  total += acc

total = total/test_num
print('FB only Accuracy: {:.4f}'.format(total))

FB only Accuracy: 0.6550


In [ ]:
model = FBGCN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.05, weight_decay=5e-5)
total = 0
test_num = 1
for i in range(0,test_num):
  model.train()
  for epoch in range(30):
      optimizer.zero_grad()
      out = model(data, adj, degree)
      loss = F.nll_loss(out[train_mask], data.y[train_mask])
      loss.backward()
      optimizer.step()
  model.eval()
  _, pred = model(data, adj, degree).max(dim=1)
  correct = int(pred[test_mask].eq(data.y[test_mask]).sum().item())
  acc = correct / int(test_mask.sum())
  total += acc
total = total/test_num
print('FBGCN Accuracy: {:.4f}'.format(total))

In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print (name, param.data)